See https://github.com/techleadhd/chatgpt-retrieval/blob/main/chatgpt.py and
[Using ChatGPT with YOUR OWN Data. This is magical. (LangChain OpenAI API)](https://youtu.be/9AXP7tCI9PI?si=Vblh5hzNao4LO_GF) from the [TechLead](https://www.youtube.com/@TechLead)

In [1]:
%%time
from langchain.chains import ConversationalRetrievalChain
# In the __init__.py of langchain/libs/langchain/langchain/indexes/
from langchain.indexes import VectorstoreIndexCreator
# This code for DirectoryLoader is in
# langchain/libs/community/langchain_community/document_loaders/directory.py
from langchain_community.document_loaders import DirectoryLoader

CPU times: user 2.35 s, sys: 608 ms, total: 2.96 s
Wall time: 2.28 s


In [2]:
from pathlib import Path
import sys

# Make this path be the project's "base" directory, so we can include modules
notebook_directory_ancestor = Path.cwd().resolve().parent.parent
print(notebook_directory_ancestor)
core_code_directory = notebook_directory_ancestor / "CoreCode/"

is_core_code_directory_in_sys_path = str(core_code_directory) in sys.path
is_notebook_directory_ancestor_in_sys_path = str(notebook_directory_ancestor) in sys.path
print("Is CoreCode directory in sys.path?", is_core_code_directory_in_sys_path)
print("Is notebook directory's ancestor in sys.path?", is_notebook_directory_ancestor_in_sys_path)

/InServiceOfX
Is CoreCode directory in sys.path? False
Is notebook directory's ancestor in sys.path? False


In [3]:
if not is_core_code_directory_in_sys_path:
    sys.path.append(str(core_code_directory))

In [4]:
from CoreCode.FileIO.get_filepaths import get_filepaths
from CoreCode.Utilities.LoadConfigurationFile import LoadConfigurationFile

In [5]:
configuration = LoadConfigurationFile.load_configuration_file()

In [6]:
print(configuration.keys())
finance_path = configuration['BASE_DATA_PATH'] / "Public" / "Finances/"
print(finance_path)
print(finance_path.exists())

dict_keys(['BASE_DATA_PATH'])
/Data/Public/Finances
True


By default, [DirectoryLoader](https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory) uses `UnstructuredLoader` class.

In [7]:
dir_loader = DirectoryLoader(path=finance_path, recursive=True, show_progress=True, use_multithreading=True)
print(dir_loader.path)
print(dir_loader.glob)
print(dir_loader.exclude)
print(dir_loader.loader_cls)

/Data/Public/Finances
**/[!.]*
()
<class 'langchain_community.document_loaders.unstructured.UnstructuredFileLoader'>


In [8]:
vector_store_index = VectorstoreIndexCreator(vectorstore_kwargs={"persist_directory":"chroma_persist"})

/ThirdParty/langchain/libs/core/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [9]:
from CoreCode.Utilities.LoadEnvironmentFile import load_environment_file
load_environment_file()

See `langchain/libs/langchain/langchain/indexes/vectorstore.py`: see that embedding, default factory, is OpenAIEmbeddings. That's why you needed the OpenAI key.

In [15]:
chroma_persist_location = configuration['BASE_DATA_PATH'] / "chroma_persist"
print(chroma_persist_location)
print(type(chroma_persist_location))
vector_store_index = VectorstoreIndexCreator(vectorstore_kwargs={"persist_directory":str(chroma_persist_location)})

/Data/chroma_persist
<class 'pathlib.PosixPath'>


In [16]:
%time
index = vector_store_index.from_loaders([dir_loader,])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs


  0%|                                                    | 0/22 [00:00<?, ?it/s]Error loading file /Data/Public/Finances/llc-12a.pdf
Error loading file /Data/Public/Finances/llc-12-112021.pdf
Error loading file /Data/Public/Finances/llc-1.pdf
Error loading file /Data/Public/Finances/llc-2-na.pdf
Error loading file /Data/Public/Finances/llc-10.pdf
Error loading file /Data/Public/Finances/llc-2.pdf
Error loading file /Data/Public/Finances/Taxes/f1099msc.pdf
Error loading file /Data/Public/Finances/Taxes/2023-540-2ez-booklet.html
Error loading file /Data/Public/Finances/Taxes/i1099gi.pdf
Error loading file /Data/Public/Finances/Taxes/i1040tt.pdf
Error loading file /Data/Public/Finances/Taxes/p5520.pdf
Error loading file /Data/Public/Finances/Taxes/2023-540-booklet.html
Error loading file /Data/Public/Finances/Taxes/f1099div.pdf
Error loading file /Data/Public/Finances/Taxes/i1040gi.pdf
Error loading file /Data/Public/Finances/Taxes/i1099div.pdf
Error loading file /Data/Public/Finances/Tax

ValueError: Expected IDs to be a non-empty list, got []

[Retrieval, Document loaders, PDF](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

In [17]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [18]:
paths = get_filepaths(finance_path, suffix=".pdf")
print(paths[:4])
loaders = []
datas = []
for path in paths:
    loader = UnstructuredPDFLoader(path)
    loaders.append(loader)
    data = loader.load()
    datas.append(data)
    
print(type(loaders[0]))
print(type(datas[0]))

[PosixPath('/Data/Public/Finances/llc-12a.pdf'), PosixPath('/Data/Public/Finances/llc-12-112021.pdf'), PosixPath('/Data/Public/Finances/llc-2-na.pdf'), PosixPath('/Data/Public/Finances/llc-1.pdf')]


AttributeError: partially initialized module 'cv2' has no attribute 'dnn' (most likely due to a circular import)

In [19]:
from langchain_community.document_loaders import UnstructuredFileLoader

In [20]:
loader = UnstructuredFileLoader(paths)
print(type(loader))
docs = loader.load()
print(type(docs))

<class 'langchain_community.document_loaders.unstructured.UnstructuredFileLoader'>


AttributeError: partially initialized module 'cv2' has no attribute 'dnn' (most likely due to a circular import)